In [6]:
import urllib
import requests
import bs4

# import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from fake_useragent import UserAgent

# 1. 댓글 수집하기

In [7]:
url = 'http://corners.auction.co.kr/AllKill/AllDay.aspx'

## 1. urllib을 이용한 html page 가져오기

In [8]:
file, http = urllib.request.urlretrieve(url, './data/auction_list.html')

- 이미지, 영상 등의 파일을 바로 저장이 가능

In [9]:
request = urllib.request.Request(url)
response = urllib.request.urlopen(request)

In [10]:
html = response.read()

## 2. requests을 이용한 html page 가져오기

In [11]:
response = requests.get(url)

In [12]:
# response.text
response.content

b'\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n <head>\r\n    <title>\xbf\xc1\xbc\xc7 - ALL KILL</title>\r\n\t<meta http-equiv="Content-type" content="text/html; charset=KSC_5601">\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge" />\r\n    <link rel="SHORTCUT ICON" href="http://pics.auction.co.kr/favicon_2011.ico"/>\r\n    <link rel="stylesheet" type="text/css" href="http://script.auction.co.kr/pc/style/css/corner_allkill.css">\r\n    <script type="text/javascript" src="http://script.auction.co.kr/common/jquery.js"></script>\r\n    <script type="text/javascript" src="http://script.auction.co.kr/common/jquery.lazyload.js"></script>\r\n    <script type="text/javascript" src="http://script.auction.co.kr/style/js/ui.header.js"></script>\r\n    <script type="text/javascript" src="http://script.auction.co.kr/style/js/ui.rvi.js"></script>\r\n    <script type="text/javascript" src="http://script.auction.co.kr/style/js/selectlayer.js"></script>\r\n\t<script type="text/javascript" src="

## 3. 리눅스에서 selenium 설정

### 1. chrome 설치
```
#> wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | sudo apt-key add -
#> sudo sh -c 'echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google.list'
#> apt-get update
#> apt-get upgrade 
#> apt-get install google-chrome-stable
#> google-chrome --version
```

### 2. webdriver 다운로드
- url: https://chromedriver.chromium.org/downloads

```
#> wget https://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_linux64.zip
#> unzip chromedriver_linux64.zip
```

## 4. selenium을 통한 html page 가져오기

In [14]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

browser = webdriver.Chrome('./data/chromedriver', options=options)

In [15]:
browser.get(url)
response = browser.page_source

browser.quit()

In [16]:
response

'<html lang="ko" class="js"><head>\n    <title>옥션 - ALL KILL</title>\n\t<meta http-equiv="Content-type" content="text/html; charset=KSC_5601">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <link rel="SHORTCUT ICON" href="http://pics.auction.co.kr/favicon_2011.ico">\n    <link rel="stylesheet" type="text/css" href="http://script.auction.co.kr/pc/style/css/corner_allkill.css">\n    <script async="" src="https://montelena.auction.co.kr/montelena.js?path=http://corners.auction.co.kr/AllKill/AllDay.aspx"></script><script async="" src="https://montelena.auction.co.kr/montelena.js?path=http://corners.auction.co.kr/AllKill/AllDay.aspx"></script><script type="text/javascript" src="http://script.auction.co.kr/common/jquery.js"></script>\n    <script type="text/javascript" src="http://script.auction.co.kr/common/jquery.lazyload.js"></script>\n    <script type="text/javascript" src="http://script.auction.co.kr/style/js/ui.header.js"></script>\n    <script type="text/javascript" s